<a href="https://colab.research.google.com/github/DevManoj19/hackathon2/blob/main/hackathon2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install didkit pyld requests nest_asyncio
!apt-get install -y libsodium-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsodium-dev is already the newest version (1.0.18-1build2).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [12]:
# === INSTALL FIRST (if not already installed) ===
# !pip install didkit pyld requests nest_asyncio
# !apt-get install -y libsodium-dev  # Linux
# !brew install libsodium            # MacOS

import didkit
import json
import requests
import asyncio
from datetime import datetime, timezone
import time
import uuid
import nest_asyncio
nest_asyncio.apply()

print("Heritage Archive DID Verification System")
print("All libraries imported successfully!")
print(f"Current time: {datetime.now(timezone.utc).isoformat()}")

# === Generate DID and Key ===
print("\n=== Generating DID and Key ===")
key = didkit.generate_ed25519_key()
did = didkit.key_to_did("key", key)

print(f"Generated DID: {did}")
print(f"Key generated successfully")
print(f"DID Type: {did.split(':')[1]}")
print(f"Key Algorithm: Ed25519")

# === Metadata Registry ===
class HeritageMetadataRegistry:
    def __init__(self):
        self.registry = {}
        self.created_at = datetime.now(timezone.utc).isoformat()
        print(f"Heritage Metadata Registry initialized at {self.created_at}")

    def register_metadata(self, did_id, metadata):
        timestamp = datetime.now(timezone.utc).isoformat()
        entry = {
            "did": did_id,
            "metadata": metadata,
            "registered_at": timestamp,
            "registry_id": str(uuid.uuid4())[:8]
        }
        self.registry[did_id] = entry
        return f"✅ Metadata registered for {did_id[:20]}... at {timestamp}"

    def lookup_metadata(self, did_id):
        return self.registry.get(did_id, "❌ No metadata found for this DID")

    def list_all_dids(self):
        return list(self.registry.keys())

    def get_registry_stats(self):
        return {
            "total_entries": len(self.registry),
            "created_at": self.created_at,
            "last_updated": datetime.now(timezone.utc).isoformat(),
            "registered_dids": len(self.list_all_dids())
        }

    def export_registry(self, filename="heritage_registry.json"):
        with open(filename, 'w') as f:
            json.dump(self.registry, f, indent=2)
        return f"📁 Registry exported to {filename}"

heritage_registry = HeritageMetadataRegistry()

# === Create Verifiable Credential ===
async def create_heritage_credential(contributor_key, contributor_did, contributor_info):
    try:
        print(f"🔐 Creating credential for: {contributor_info.get('name', 'Unknown')}")
        verification_method = await didkit.key_to_verification_method("key", contributor_key)
        proof_options = {
            "proofPurpose": "assertionMethod",
            "verificationMethod": verification_method
        }
        current_time = datetime.now(timezone.utc).isoformat()
        expiration_time = "2026-12-31T23:59:59Z"
        credential = {
            "@context": [
                "https://www.w3.org/2018/credentials/v1",
                {
                    "HeritageContributor": "https://heritage.org/vocab#HeritageContributor",
                    "contribution": "https://heritage.org/vocab#contribution",
                    "location": "https://heritage.org/vocab#location"
                }
            ],
            "type": ["VerifiableCredential", "HeritageContributorCredential"],
            "id": f"https://heritage.org/credentials/{uuid.uuid4()}",
            "issuer": contributor_did,
            "issuanceDate": current_time,
            "expirationDate": expiration_time,
            "credentialSubject": {
                "id": contributor_did,
                "type": "HeritageContributor",
                "name": contributor_info.get("name", "Unknown Contributor"),
                "role": "Heritage Archive Contributor",
                "location": contributor_info.get("location", "Unknown Location"),
                "contribution": contributor_info.get("contribution", "Heritage preservation"),
                "expertise": contributor_info.get("expertise", "General heritage work"),
                "verified": True
            }
        }

        signed_credential = await didkit.issue_credential(
            json.dumps(credential),
            json.dumps(proof_options),
            contributor_key
        )

        verification_result = await didkit.verify_credential(signed_credential, "{}")
        verification_data = json.loads(verification_result)

        if verification_data.get('verified', False):
            print("✅ Credential verification: PASSED")
        else:
            print("❌ Credential verification: FAILED")
            if 'errors' in verification_data:
                print(f"Errors: {verification_data['errors']}")

        return {
            "signed_credential": signed_credential,
            "verification_result": verification_result,
            "credential_id": credential["id"],
            "status": "success"
        }

    except Exception as e:
        print(f"❌ Error creating credential: {str(e)}")
        return {
            "error": str(e),
            "status": "failed"
        }

# === Main Async Function ===
async def main():
    print("\n🚀 Starting Heritage Archive DID System...")

    contributor = {
        "name": "Dr. Priya Sharma",
        "location": "Kolkata, West Bengal",
        "contribution": "Digitization of 19th-century Bengali manuscripts",
        "expertise": "Bengali literature and paleography"
    }

    result = await create_heritage_credential(key, did, contributor)
    if result["status"] == "success":
        print("🎉 Credential created and verified!")
        metadata = {
            "name": contributor["name"],
            "location": contributor["location"],
            "contribution": contributor["contribution"],
            "expertise": contributor["expertise"],
            "created_at": datetime.now(timezone.utc).isoformat(),
            "credential_id": result["credential_id"]
        }
        reg = heritage_registry.register_metadata(did, metadata)
        print(reg)

        print("\n📊 Registry Stats:")
        print(json.dumps(heritage_registry.get_registry_stats(), indent=2))

        print("\n📋 Contributor Metadata:")
        print(json.dumps(heritage_registry.lookup_metadata(did), indent=2))

        print("\n📁 Exporting registry:")
        print(heritage_registry.export_registry())

    else:
        print(f"❌ Failed: {result['error']}")

# === Run It (This works in Notebooks too!) ===
await main()


Heritage Archive DID Verification System
All libraries imported successfully!
Current time: 2025-07-06T16:27:28.193465+00:00

=== Generating DID and Key ===
Generated DID: did:key:z6MkkhckZfb6gCdrVBjTCd3srETkBQMVMdJf1P63KFh2N9SS
Key generated successfully
DID Type: key
Key Algorithm: Ed25519
Heritage Metadata Registry initialized at 2025-07-06T16:27:28.195012+00:00

🚀 Starting Heritage Archive DID System...
🔐 Creating credential for: Dr. Priya Sharma
❌ Error creating credential: Expansion failed: Expansion failed: Key expansion failed
❌ Failed: Expansion failed: Expansion failed: Key expansion failed
